---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns = open("university_towns.txt", "r")
    
    university_towns_df = pd.DataFrame(columns = ["State", "RegionName"])
    
    lines = university_towns.readlines()
    for i in range(0, len(lines)):
        line = lines[i]
        if "[edit]" in line:
            state = line
            x = line
            regionName = lines[i+1]
            university_towns_df.loc[i] = [state, regionName]
    
        else:
            regionName = line
            state = x
            university_towns_df.loc[i] = [state, regionName]
    
    university_towns_df.drop_duplicates(subset = "RegionName", keep = "first", inplace = True)
    
    university_towns_df = university_towns_df.reset_index(drop = True)
    
    university_towns_df['State'] = university_towns_df['State'].str.split('[', 1).str[0]
    university_towns_df['RegionName'] = university_towns_df['RegionName'].str.split('(', 1).str[0]
    university_towns_df['RegionName'] = university_towns_df['RegionName'].str.rstrip()

    return university_towns_df
   
    
    
get_list_of_university_towns()


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    recession_start_df = pd.read_excel('gdplev.xls', skiprows=5, usecols = [4,5,6]).drop([0,1], axis=0).drop(['GDP in billions of current dollars.1'], axis = 1).reset_index(drop=True)
    
    recession_start_df = recession_start_df.rename(columns = {'Unnamed: 4': 'YearQ', 'GDP in billions of chained 2009 dollars.1': 'GDP in billions of chained 2009 dollars'})
    
    recession_start_df = recession_start_df[~recession_start_df.YearQ.str.startswith("19")]
    
    for i, j in recession_start_df.iterrows():
        if (recession_start_df.loc[i+1, "GDP in billions of chained 2009 dollars" ] - recession_start_df.loc[i, "GDP in billions of chained 2009 dollars"] < 0) & (recession_start_df.loc[i+2, "GDP in billions of chained 2009 dollars" ] - recession_start_df.loc[i+1, "GDP in billions of chained 2009 dollars"] < 0):
            break
    return recession_start_df.loc[i+1, "YearQ"]

get_recession_start()



'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    recession_end_df = pd.read_excel('gdplev.xls', skiprows=5, usecols = [4,5,6]).drop([0,1], axis=0).drop(['GDP in billions of current dollars.1'], axis = 1).reset_index(drop=True)
    
    recession_end_df = recession_end_df.rename(columns = {'Unnamed: 4': 'YearQ', 'GDP in billions of chained 2009 dollars.1': 'GDP in billions of chained 2009 dollars'})
    
    recession_end_df = recession_end_df[~recession_end_df.YearQ.str.startswith("19")]
    
    for i, j in recession_end_df.iterrows():
        if (recession_end_df.loc[i+1, "GDP in billions of chained 2009 dollars" ] - recession_end_df.loc[i, "GDP in billions of chained 2009 dollars"] < 0) & (recession_end_df.loc[i+2, "GDP in billions of chained 2009 dollars" ] - recession_end_df.loc[i+1, "GDP in billions of chained 2009 dollars"] < 0):
            x = i
            break
            
    for a, b in recession_end_df.loc[x:].iterrows():
        if (recession_end_df.loc[a+1, "GDP in billions of chained 2009 dollars" ] - recession_end_df.loc[a, "GDP in billions of chained 2009 dollars"] > 0) & (recession_end_df.loc[a+2, "GDP in billions of chained 2009 dollars" ] - recession_end_df.loc[a+1, "GDP in billions of chained 2009 dollars"] > 0):
            break
    return recession_end_df.loc[a+2, "YearQ"]

get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    recession_bottom_df = pd.read_excel('gdplev.xls', skiprows=5, usecols = [4,5,6]).drop([0,1], axis=0).drop(['GDP in billions of current dollars.1'], axis = 1).reset_index(drop=True)
    
    recession_bottom_df = recession_bottom_df.rename(columns = {'Unnamed: 4': 'YearQ', 'GDP in billions of chained 2009 dollars.1': 'GDP in billions of chained 2009 dollars'})
    
    recession_bottom_df = recession_bottom_df[~recession_bottom_df.YearQ.str.startswith("19")]
    
    for i, j in recession_bottom_df.iterrows():
        if (recession_bottom_df.loc[i+1, "GDP in billions of chained 2009 dollars" ] - recession_bottom_df.loc[i, "GDP in billions of chained 2009 dollars"] < 0) & (recession_bottom_df.loc[i+2, "GDP in billions of chained 2009 dollars" ] - recession_bottom_df.loc[i+1, "GDP in billions of chained 2009 dollars"] < 0):
            x = i
            break
            
    for a, b in recession_bottom_df.loc[x:].iterrows():
        if (recession_bottom_df.loc[a+1, "GDP in billions of chained 2009 dollars" ] - recession_bottom_df.loc[a, "GDP in billions of chained 2009 dollars"] > 0) & (recession_bottom_df.loc[a+2, "GDP in billions of chained 2009 dollars" ] - recession_bottom_df.loc[a+1, "GDP in billions of chained 2009 dollars"] > 0):
            y= a + 2
            break
            
    recession_bottom_df = recession_bottom_df.loc[x:y]
    
    x = recession_bottom_df["GDP in billions of chained 2009 dollars"].idxmin()
    
    ANSWER = recession_bottom_df.loc[x]['YearQ']
    
    return ANSWER

get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    
    housing_data_quarters_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    housing_data_quarters_df = housing_data_quarters_df.loc[:, ~housing_data_quarters_df.columns.str.startswith("19")].drop(['RegionID', "Metro", "CountyName", "SizeRank"], axis = 1) #drop unnecesary columns
    
    housing_data_quarters_df['State'] = housing_data_quarters_df['State'].replace(states)
    
    housing_data_quarters_df = housing_data_quarters_df.set_index(['State', 'RegionName'])
    
    housing_data_quarters_df["2016-09"]=0
    
    housing_data_quarters_df2 = housing_data_quarters_df.drop(housing_data_quarters_df.columns, axis=1)
    
    i= 3
    while i < 202: 
        x = str(((i/3)-1))
        housing_data_quarters_df2[x] = (((housing_data_quarters_df.iloc[:, i-3])+(housing_data_quarters_df.iloc[:, i-2])+(housing_data_quarters_df.iloc[:, i-1]))/3)
        i+=3
        
    housing_data_quarters_df3 = pd.read_excel('gdplev.xls', skiprows=5, usecols = [4,5,6]).drop([0,1], axis=0).drop(['GDP in billions of current dollars.1'], axis = 1).reset_index(drop=True)
    
    housing_data_quarters_df3 = housing_data_quarters_df3.rename(columns = {'Unnamed: 4': 'YearQ', 'GDP in billions of chained 2009 dollars.1': 'GDP in billions of chained 2009 dollars'})
    
    housing_data_quarters_df3 = housing_data_quarters_df3[~housing_data_quarters_df3.YearQ.str.startswith("19")]
    
    housing_data_quarters_df3.loc[66, "YearQ"] = "2016q3"
    
    housing_data_quarters_df3 = housing_data_quarters_df3.reset_index(drop = True)
        
    housing_data_quarters_df2.columns = housing_data_quarters_df3["YearQ"].tolist()
    
    
    return housing_data_quarters_df2.head(50)


convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,391466.666667
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,389366.666667
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,141333.333333
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,85800.000000
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,130133.333333
Nevada,Las Vegas,132600.000000,134366.666667,135400.000000,137000.000000,139533.333333,141733.333333,143366.666667,146133.333333,149333.333333,150933.333333,...,170066.666667,173400.000000,175466.666667,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,133300.000000
California,San Diego,222900.000000,234366.666667,245433.333333,256033.333333,267200.000000,276266.666667,284500.000000,291933.333333,301233.333333,312866.666667,...,480200.000000,489033.333333,496433.333333,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,359833.333333
Texas,Dallas,84466.666667,83866.666667,84866.666667,87833.333333,89733.333333,89300.000000,89066.666667,90900.000000,92566.666667,93800.000000,...,106633.333333,108900.000000,111533.333333,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,99533.333333
California,San Jose,374266.666667,406566.666667,431866.666667,455500.000000,470666.666667,470200.000000,456800.000000,445566.666667,441433.333333,457766.666667,...,679400.000000,697033.333333,714933.333333,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,548133.333333


In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
   
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', -1)
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

    
    housing_data_quarters_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    housing_data_quarters_df = housing_data_quarters_df.loc[:, ~housing_data_quarters_df.columns.str.startswith("19")].drop(['RegionID', "Metro", "CountyName", "SizeRank"], axis = 1) #drop unnecesary columns
    
    housing_data_quarters_df['State'] = housing_data_quarters_df['State'].replace(states)
    
    housing_data_quarters_df = housing_data_quarters_df.set_index(['State', 'RegionName'])
    
    housing_data_quarters_df["2016-09"]=0
    
    housing_data_quarters_df2 = housing_data_quarters_df.drop(housing_data_quarters_df.columns, axis=1)
    
    i= 3
    while i < 202: 
        x = str(((i/3)-1))
        housing_data_quarters_df2[x] = (((housing_data_quarters_df.iloc[:, i-3])+(housing_data_quarters_df.iloc[:, i-2])+(housing_data_quarters_df.iloc[:, i-1]))/3)
        i+=3
        
    housing_data_quarters_df3 = pd.read_excel('gdplev.xls', skiprows=5, usecols = [4,5,6]).drop([0,1], axis=0).drop(['GDP in billions of current dollars.1'], axis = 1).reset_index(drop=True)
    
    housing_data_quarters_df3 = housing_data_quarters_df3.rename(columns = {'Unnamed: 4': 'YearQ', 'GDP in billions of chained 2009 dollars.1': 'GDP in billions of chained 2009 dollars'})
    
    housing_data_quarters_df3 = housing_data_quarters_df3[~housing_data_quarters_df3.YearQ.str.startswith("19")]
    
    housing_data_quarters_df3.loc[66, "YearQ"] = "2016q3"
    
    housing_data_quarters_df3 = housing_data_quarters_df3.reset_index(drop = True)
        
    housing_data_quarters_df2.columns = housing_data_quarters_df3["YearQ"].tolist()
    
    housing_price_recession_decline = pd.DataFrame(housing_data_quarters_df2["2009q2"] - housing_data_quarters_df2["2008q2"])
  
    housing_price_recession_decline.columns = ['Housing Price Decline During Recession']
    
    university_towns = open("university_towns.txt", "r")
    
    university_towns_df = pd.DataFrame(columns = ["State", "RegionName"])
    
    lines = university_towns.readlines()
    for i in range(0, len(lines)):
        line = lines[i]
        if "[edit]" in line:
            state = line
            x = line
            regionName = lines[i+1]
            university_towns_df.loc[i] = [state, regionName]
    
        else:
            regionName = line
            state = x
            university_towns_df.loc[i] = [state, regionName]
    
    university_towns_df.drop_duplicates(subset = "RegionName", keep = "first", inplace = True)
    
    university_towns_df = university_towns_df.reset_index(drop = True)
    
    university_towns_df['State'] = university_towns_df['State'].str.split('[', 1).str[0]
    university_towns_df['RegionName'] = university_towns_df['RegionName'].str.split('(', 1).str[0]
    
    university_cities = university_towns_df['RegionName'].str.rstrip().values.tolist()
    
   # housing_price_recession_decline[city] = 
    
    university_towns_housing_price_recession_decline = housing_price_recession_decline[housing_price_recession_decline.index.get_level_values('RegionName').isin(university_cities)]
    
    not_university_towns_housing_price_recession_decline = housing_price_recession_decline[~housing_price_recession_decline.index.get_level_values('RegionName').isin(university_cities)]
    
    ttest = ttest_ind(university_towns_housing_price_recession_decline.dropna()['Housing Price Decline During Recession'], not_university_towns_housing_price_recession_decline.dropna()['Housing Price Decline During Recession'])
    
    different = False
    if ttest.statistic != 0 and ttest.pvalue < 0.01:
        different = True
        pvalue = ttest.pvalue
        
        if ttest.statistic > 0:
            better = "university town"
        else:
            better = "non-university town"
    
    return  (different, pvalue, better)

    
run_ttest()

(True, 1.1693116003332356e-06, 'university town')